In [111]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold

import pickle


## Import Files

In [132]:
file = open("sepos/seposAug.txt", "rb")
df = pickle.load(file)

In [138]:
# Dataframe shuffeln

df = df.sample(frac=1, random_state=123) # frac meint dabei alle Rows werden behalten beim shuffeln

In [140]:
# für den Split müss dass Dataframe in X (Daten) und y (labels) geteilt werden 

X = df.text
y = df.label

In [141]:
X[0:3]

1361    Frau Brickner leistet mit ihren Kommentaren we...
838      Bis auf ganz wenige Ausnahmen sind Männer kör...
1855     Gegen dieses Abdullah Zentrum gabs sehr wohl ...
Name: text, dtype: object

In [142]:
y[0:3]

1361    0
838     0
1855    0
Name: label, dtype: int64

## Stratified10-Fold-Cross-Validation

In [143]:
# setup the split

#skf = StratifiedKFold(n_splits=10, shuffle=True) # make 10 Splits

#hier gschuffelt, mit seed. Kann nachvollzogen werden. 



skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)

#auf Basis der X y 
skf.get_n_splits(X, y)



10

In [144]:
# nun das Dataframe anhand der Splits aufteilen:

kftrain_list = [] # Liste mit den Indices der einzelen Folds
kftest_list = []

for train_index, test_index in skf.split(X,y):
    kftrain_list.append(train_index) # gibt mir eine Liste mit Listen
    kftest_list.append(test_index)
 

In [145]:
# check if it worked, 

set(kftrain_list[9]) & set(kftest_list[9])

# wenn ein leeres Set zurückkommt bedeutet dass, die listen für die Folds stimmen. 


set()

In [146]:
# nun die Daten zu den Indices der Trainingsdaten für jeden Fold zuordnen: 

train_folds={}

counter = 0
for i in kftrain_list:
    train_folds["train_fold_{}".format(counter)] = df[df.index.isin(i)]
    counter +=1


In [148]:

test_folds = {}

counter = 0
for i in kftest_list:
    test_folds["test_fold_{}".format(counter)] = df[df.index.isin(i)]
    counter += 1


In [149]:
test_folds["test_fold_0"].label.sum()

4

In [150]:
set(kftrain_list[9]) & set(kftest_list[9])

set()

In [151]:
# Kontrollschritt

set(train_folds["train_fold_0"]["id"]) & set(test_folds["test_fold_0"]["id"])



set()

In [152]:
# für alle Folds Trainingsdaten mit augemntierten DAten zusammenfügen

def TenFoldTrain (df, include):
    
    """
    Ein Dataframe eines Folds wird übergeben, die augmentierten Posts werden hinzugefügt
    in Frames muss dann händisch festgelegt werden, ob und welche augmentierte Daten hinzukommen
    
    """
    
    
    ttrain_syn = df[['id', 'label', 'alpha', "text_syn"]] #nimmt nur synonym replaced Post
    print("mit 1 gelabelte Posts:", ttrain_syn.label.sum())
    print("Posts insgesamt", ttrain_syn.label.count())
    ttrain_syn = ttrain_syn.dropna() # schmeisst NaNs raus, bekomme ein df mit nur syno augementierten Posts
    ttrain_syn = ttrain_syn.rename(columns={"text_syn": "text"}) # Umbbennnenen der Augementierten Spalte in Text damit sich dass dann leichter concaten lässt.
    
    ttrain_trans = df[['id', 'label', 'alpha', "ru-back-yandex"]]
    print("mit 1 gelabelte Posts:", ttrain_trans.label.sum())
    print("Posts insgesamt", ttrain_trans.label.count())
    ttrain_trans = ttrain_trans.dropna()
    ttrain_trans = ttrain_trans.rename(columns= {"ru-back-yandex": "text"})
    
    # jetzt noch das Dataframe mit den Unbehandelten Daten:
    ttrain_1 = df.drop(columns= ["text_syn", "ru-back-yandex"])
    
    #alles in ein dataframe zusammenfassen:
    # HIER ÄNDERN UM EIN UNAUGMENTIERTES GEFOLDETES DF; EIN SYN DF UND EIN TRANS FRAME ZU ERHALTEN. 
    
    if include == 1:
        
        #nur die unaugmentierten Trainingsdaten
        frames = [ttrain_1]
    if include == 2:
        
        #inkludier die synonym replaced posts 
        frames = [ttrain_1, ttrain_syn]
        
    if include ==3:
        # inkludiert die rückübersetzten
        frames = [ttrain_1, ttrain_trans]
    
    if include == 4:
        
        #oringalposts plus synorep plus backtrans posts
        frames = [ttrain_1, ttrain_syn, ttrain_trans]
        
    else:
        print("Option 1-4 wählbar")

    
    fin_train = pd.concat(frames) # hier war der Fehler... frames1 statt frames ;-)
    
    print("na vor drop",fin_train.isna().sum())
    fin_train = fin_train.dropna()
    print("na nach drop", fin_train.isna().sum)
    
    
    # noch shuffeln: 
    #fin_train1 = fin_train.sample(frac=1, random_state=123).reset_index(drop=True)
    fin_train1 = fin_train.reset_index(drop=True)
    
    #das id-Feld zurücksetzten:
    fin_train1["id"] = range(len(fin_train1))
    
    return fin_train1

In [153]:
def TenFoldTest(df):
    
    """
    hier werden die Test Folds um ihre Spalten bereinigt, da ja augmentierten Daten in den Test sets nichts verloren haben
    """
    
    fin_test = df.drop(columns = ["text_syn", "ru-back-yandex"])
    
    # shuffeln
    #fin_test1 = fin_test.sample(frac=1, random_state = 123).reset_index(drop=True)
    fin_test1 = fin_test.reset_index(drop=True)
    
    #das id-Feld zurücksetzten:
    fin_test1["id"] = range(len(fin_test1))
     
    
    return fin_test1


## Folds erstellen

In [154]:
# Train

dcc_folds_train = {}

counter=0
for k, v in train_folds.items():
    dcc_folds_train["{}_fold".format(counter)] = TenFoldTrain(v, 4)
    counter += 1


mit 1 gelabelte Posts: 39
Posts insgesamt 3194
mit 1 gelabelte Posts: 39
Posts insgesamt 3194
na vor drop id       0
label    0
alpha    0
text     0
dtype: int64
na nach drop <bound method DataFrame.sum of          id  label  alpha   text
1361  False  False  False  False
838   False  False  False  False
1855  False  False  False  False
2825  False  False  False  False
553   False  False  False  False
791   False  False  False  False
1942  False  False  False  False
2312  False  False  False  False
742   False  False  False  False
2716  False  False  False  False
2024  False  False  False  False
2855  False  False  False  False
1984  False  False  False  False
1082  False  False  False  False
1430  False  False  False  False
222   False  False  False  False
2763  False  False  False  False
2759  False  False  False  False
3456  False  False  False  False
929   False  False  False  False
1191  False  False  False  False
3130  False  False  False  False
2048  False  False  False  False
1

[3257 rows x 4 columns]>
mit 1 gelabelte Posts: 42
Posts insgesamt 3195
mit 1 gelabelte Posts: 42
Posts insgesamt 3195
na vor drop id       0
label    0
alpha    0
text     0
dtype: int64
na nach drop <bound method DataFrame.sum of          id  label  alpha   text
1361  False  False  False  False
2825  False  False  False  False
553   False  False  False  False
791   False  False  False  False
1942  False  False  False  False
2312  False  False  False  False
742   False  False  False  False
2716  False  False  False  False
2024  False  False  False  False
2855  False  False  False  False
1984  False  False  False  False
1082  False  False  False  False
1027  False  False  False  False
1430  False  False  False  False
222   False  False  False  False
2763  False  False  False  False
2759  False  False  False  False
1681  False  False  False  False
3456  False  False  False  False
929   False  False  False  False
1191  False  False  False  False
3130  False  False  False  False
2048  Fal

In [170]:
dcc_folds_test = {}

counter=0
for k, v in test_folds.items():
    dcc_folds_test["{}_fold".format(counter)] = TenFoldTest(v)
    counter += 1

## exportier die folds


In [705]:
file = open("The_10_Folds/seneu_train_4.txt", "wb")
pickle.dump(dcc_folds_train, file)
file.close()

In [706]:
file = open("The_10_Folds/seneu_test_4.txt", "wb")
pickle.dump(dcc_folds_test, file)
file.close()